In [22]:
import cv2 #opencv
import mediapipe as mp
import math
import csv
import datetime
import pandas as pd
import os
from mediapipe_estimate import estimate
pd.options.mode.chained_assignment = None 

In [23]:
directory_path = 'result_videos/'

In [24]:
test_dat=data=pd.read_csv('golfDB_front_orig.csv',index_col='id')
test_dat=test_dat[['youtube_id','events_2','events_5','events_7']]
test_dat['events_5']=test_dat['events_5']-test_dat['events_2']+15
test_dat['events_7']=test_dat['events_7']-test_dat['events_2']+15

test_dat=test_dat.drop(['events_2'],axis=1)
test_dat

,youtube_id,events_5,events_7
id,,,
8,-M5SITXMA2Y,45,53
10,gOBVh7fzyZo,39,47
14,vN3Uc_EhnnY,41,49
19,iW323nsTGtU,49,57
21,xD6KDqPF9cc,39,45
...,...,...,...
1370,g90r9cs1tTw,37,43
1372,eNlBbMI-MhQ,49,59
1374,6K1FD0sOwoI,54,63


In [29]:
name_list=list('output_video_'+pd.unique(test_dat.youtube_id))
all=len(name_list)
correct_contact=0
correct_top=0
correct_address=0
error_frames=[]

swing_part='top'
folder_name = 'frames_'+swing_part
full_path = os.path.join(directory_path, folder_name)
if not os.path.exists(full_path):
    os.makedirs(full_path)


for fl in name_list:
    try:
        video_files = [file for file in os.listdir(directory_path) if fl in file and file.endswith('.mp4')]
        video_path = directory_path + video_files[0]
        data = pd.read_csv('results_wrist_elbow/variables_' + fl + '.csv')
        cap = cv2.VideoCapture(video_path)
        #consider only time between backswing and finish
        halfway_back_ind=data['right_wrist_x'].idxmin()
        halfway_front_ind=data.right_wrist_x[data.index>halfway_back_ind].idxmax()
        middle_data=data[(data.index>halfway_back_ind)&(data.index<halfway_front_ind)]

        #find moment of ball contact as the lowest wrist point on y
        contact_frame=middle_data[middle_data['right_wrist_y']==middle_data['right_wrist_y'].max()].index
        
        #isolate only backswing data
        back_data=data[(data.index<contact_frame.item())]

        #find moment of top of backswing as the highest wrist point on y
        top_backswing_frame=back_data[back_data['right_wrist_y']==back_data['right_wrist_y'].min()].index

        #find moment before start of the swing as the lowest wrist point on y before going halfway back
        halfway_back_data=data[data['right_wrist_x'].index<halfway_back_ind]
        address_frame=halfway_back_data[halfway_back_data['right_wrist_y']==halfway_back_data['right_wrist_y'].max()].index
        
        #extract the gt from golfDB
        yt_id='_'.join(fl.split('_')[2::])
        gt_contact=test_dat[test_dat.youtube_id==yt_id].events_7.item()
        gt_top=test_dat[test_dat.youtube_id==yt_id].events_5.item()
        gt_address=15

        #calculte tolerance as per golfDB paper (fps=30)
        tolerance_hit=2*(gt_contact//30)
        tolerance_top=2*(gt_top//30)
        tolerance_address=2*(gt_address//30)
        

        

        # Processing based on swing_part value
        if swing_part == 'address':
            swing_part_id=max([address_frame.item()-3,0])
            cap.set(cv2.CAP_PROP_POS_FRAMES, swing_part_id)
            ret, frame = cap.read()
            frame_path = os.path.splitext(video_path)[0].split("/")[0] + '/frames_address/' + os.path.splitext(video_path)[0].split("/")[1] + f'_frame_{address_frame.item()}.jpg'
            cv2.imwrite(frame_path, frame)
            cap.release()
            if gt_address - tolerance_address <= address_frame.item() <= tolerance_address + gt_address:
                correct_address += 1

        elif swing_part == 'contact':
            swing_part_id=contact_frame.item()
            cap.set(cv2.CAP_PROP_POS_FRAMES, swing_part_id)
            ret, frame = cap.read()
            frame_path = os.path.splitext(video_path)[0].split("/")[0] + '/frames_contact/' + os.path.splitext(video_path)[0].split("/")[1] + f'_frame_{contact_frame.item()}.jpg'

            cv2.imwrite(frame_path, frame)
            cap.release()
            if gt_contact - tolerance_hit <= contact_frame.item() <= tolerance_hit + gt_contact:
                correct_contact += 1

        elif swing_part == 'top':
            swing_part_id=top_backswing_frame.item()
            cap.set(cv2.CAP_PROP_POS_FRAMES, swing_part_id)
            ret, frame = cap.read()
            frame_path = os.path.splitext(video_path)[0].split("/")[0] + '/frames_top/' + os.path.splitext(video_path)[0].split("/")[1] + f'_frame_{top_backswing_frame.item()}.jpg'

            cv2.imwrite(frame_path, frame)
            cap.release()
            if gt_top-tolerance_top<=top_backswing_frame.item()<=tolerance_top+gt_top:
                correct_top+=1

    except Exception as e:
        print(e, fl)
        print('----------------------------------------------------------------------------------------------------')
        all=all-1

if swing_part == 'address':
    accuracy_address=correct_address/all
    print(accuracy_address)
elif swing_part == 'contact':
    accuracy_contact=correct_contact/all
    print(accuracy_contact)
elif swing_part == 'top':
    accuracy_top=correct_top/all
    print(accuracy_top)


list index out of range output_video_vN3Uc_EhnnY
----------------------------------------------------------------------------------------------------
can only convert an array of size 1 to a Python scalar output_video_j6h-3Ez0714
----------------------------------------------------------------------------------------------------
list index out of range output_video_8n9BBNEj3yk
----------------------------------------------------------------------------------------------------
list index out of range output_video_FPvudrvNw6A
----------------------------------------------------------------------------------------------------
list index out of range output_video_bF8A2zdAfAs
----------------------------------------------------------------------------------------------------
can only convert an array of size 1 to a Python scalar output_video_04d08bM6-6U
----------------------------------------------------------------------------------------------------
list index out of range output_video_0

In [26]:
correct_address

4